# 怎样实现一个键对应多个值的字典（也叫 multidict）？

选择使用列表还是集合取决于你的实际需求。如果你想保持元素的插入顺序就应该使用列表， 如果想去掉重复元素就使用集合（并且不关心元素的顺序问题）
选择使用列表还是集合取决于你的实际需求。如果你想保持元素的插入顺序就应该使用列表， 如果想去掉重复元素就使用集合（并且不关心元素的顺序问题）
如果使用 defaultdict 的话代码就更加简洁了：

d = defaultdict(list)
for key, value in pairs:
    d[key].append(value)

In [2]:
d = {
    'a' : [1, 2, 3],
    'b' : [4, 5]
}
e = {
    'a' : {1, 2, 3},
    'b' : {4, 5}
}

from collections import defaultdict

d = defaultdict(list)
d['a'].append(1)
d['a'].append(2)
d['b'].append(4)
print(d)

d = defaultdict(set)
d['a'].add(1)
d['a'].add(2)
d['b'].add(4)

print(d)

defaultdict(<class 'list'>, {'a': [1, 2], 'b': [4]})
defaultdict(<class 'set'>, {'a': {1, 2}, 'b': {4}})


需要注意的是， defaultdict 会自动为将要访问的键（就算目前字典中并不存在这样的键）创建映射实体。 如果你并不需要这样的特性，你可以在一个普通的字典上使用 setdefault() 方法来代替, 但是很多程序员觉得 setdefault() 用起来有点别扭。因为每次调用都得创建一个新的初始值的实例

In [3]:
d = {} # 一个普通的字典
d.setdefault('a', []).append(1)
d.setdefault('a', []).append(2)
d.setdefault('b', []).append(4)
print(d)

{'a': [1, 2], 'b': [4]}


# 你想创建一个字典，并且在迭代或序列化这个字典的时候能够控制元素的顺序
OrderedDict 内部维护着一个根据键插入顺序排序的双向链表。每次当一个新的元素插入进来的时候， 它会被放到链表的尾部。对于一个已经存在的键的重复赋值不会改变键的顺序, 需要注意的是，一个 OrderedDict 的大小是一个普通字典的两倍, 你就得仔细权衡一下是否使用 OrderedDict 带来的好处要大过额外内存消耗的影响。

In [4]:
from collections import OrderedDict

d = OrderedDict()
d['foo'] = 1
d['bar'] = 2
d['spam'] = 3
d['grok'] = 4
# Outputs "foo 1", "bar 2", "spam 3", "grok 4"
for key in d:
    print(key, d[key])

foo 1
bar 2
spam 3
grok 4


当你想要构建一个将来需要序列化或编码成其他格式的映射的时候， OrderedDict 是非常有用的。 比如，你想精确控制以 JSON 编码后字段的顺序，你可以先使用 OrderedDict 来构建这样的数据

In [5]:
import json
json.dumps(d)

'{"foo": 1, "bar": 2, "spam": 3, "grok": 4}'

# 怎样在数据字典中执行一些计算操作（比如求最小值、最大值、排序等等）？
为了对字典值执行计算操作，通常需要使用 zip() 函数先将键和值反转过来,需要注意的是 zip() 函数创建的是一个只能访问一次的迭代器 

In [7]:
prices = {
    'ACME': 45.23,
    'AAPL': 612.78,
    'IBM': 205.55,
    'HPQ': 37.20,
    'FB': 10.75
}

min_price = min(zip(prices.values(), prices.keys()))
# min_price is (10.75, 'FB')
max_price = max(zip(prices.values(), prices.keys()))
# max_price is (612.78, 'AAPL')

print(min_price)
print(max_price)

prices_sorted = sorted(zip(prices.values(), prices.keys()))
print(prices_sorted)

min_com = min(prices, key=lambda k: prices[k]) # Returns 'FB'
max_com = max(prices, key=lambda k: prices[k]) # Returns 'AAPL'
print(min_com)
print(max_com)

(10.75, 'FB')
(612.78, 'AAPL')
[(10.75, 'FB'), (37.2, 'HPQ'), (45.23, 'ACME'), (205.55, 'IBM'), (612.78, 'AAPL')]
FB
AAPL


# 怎样在两个字典中寻寻找相同点（比如相同的键、相同的值等等）？

In [9]:
a = {
    'x' : 1,
    'y' : 2,
    'z' : 3
}

b = {
    'w' : 10,
    'x' : 11,
    'y' : 2
}

# Find keys in common
print(a.keys() & b.keys()) # { 'x', 'y' }
# Find keys in a that are not in b
print(a.keys() - b.keys()) # { 'z' }
# Find (key,value) pairs in common
print(a.items() & b.items()) # { ('y', 2) }

# Make a new dictionary with certain keys removed
c = {key:a[key] for key in a.keys() - {'z', 'w'}}
# c is {'x': 1, 'y': 2}
print(c)

{'x', 'y'}
{'z'}
{('y', 2)}
{'x': 1, 'y': 2}


# 怎样在一个序列上面保持元素顺序的同时消除重复的值？
比如，如果如果你想读取一个文件，消除重复行，你可以很容易像这样做：
with open(somefile,'r') as f:
for line in dedupe(f):
    ...

In [10]:
def dedupe(items, key=None):
    seen = set()
    for item in items:
        val = item if key is None else key(item)
        if val not in seen:
            yield item
            seen.add(val)

a = [1, 5, 2, 1, 9, 1, 5, 10]
print(list(dedupe(a)))

b = [ {'x':1, 'y':2}, {'x':1, 'y':3}, {'x':1, 'y':2}, {'x':2, 'y':4}]
print(list(dedupe(b, key=lambda d: (d['x'],d['y']))))

print(list(dedupe(b, key=lambda d: d['x'])))



[1, 5, 2, 9, 10]
[{'x': 1, 'y': 2}, {'x': 1, 'y': 3}, {'x': 2, 'y': 4}]
[{'x': 1, 'y': 2}, {'x': 2, 'y': 4}]


# 你的程序已经出现一大堆已无法直视的硬编码切片下标，然后你想清理下代码
假定你有一段代码要从一个记录字符串中几个固定位置提取出特定的数据字段（比如文件或类似格式）：

######    0123456789012345678901234567890123456789012345678901234567890'
record = '....................100 .......513.25 ..........'
cost = int(record[20:23]) * float(record[31:37])
与其那样写，为什么不想这样命名切片呢：

SHARES = slice(20, 23)
PRICE = slice(31, 37)
cost = int(record[SHARES]) * float(record[PRICE])


In [13]:
items = [0, 1, 2, 3, 4, 5, 6]
a = slice(2, 4)
print(a)
print(items[a])
items[a] = [10,11]
print(items)
del items[a]
print(items)


slice(2, 4, None)
[2, 3]
[0, 1, 10, 11, 4, 5, 6]
[0, 1, 4, 5, 6]


In [14]:
#如果你有一个切片对象a，你可以分别调用它的 a.start , a.stop , a.step
a = slice(5, 50, 2)
print(a.start)
print(a.stop)
print(a.step)

5
50
2


#你还能通过调用切片的 indices(size) 方法将它映射到一个确定大小的序列上， 这个方法返回一个三元组 (start, stop, step) ，所有值都会被合适的缩小以满足边界限制， 从而使用的时候避免出现 IndexError 异常

In [16]:
s = 'HelloWorld'
print(a.indices(len(s)))
for i in range(*a.indices(len(s))):
     print(s[i])

(5, 10, 2)
W
r
d


# 怎样找出一个序列中出现次数最多的元素呢？
collections.Counter 类就是专门为这类问题而设计的， 它甚至有一个有用的 most_common() 方法直接给了你答案

In [18]:
words = [
    'look', 'into', 'my', 'eyes', 'look', 'into', 'my', 'eyes',
    'the', 'eyes', 'the', 'eyes', 'the', 'eyes', 'not', 'around', 'the',
    'eyes', "don't", 'look', 'around', 'the', 'eyes', 'look', 'into',
    'my', 'eyes', "you're", 'under'
]
from collections import Counter
word_counts = Counter(words)
# 出现频率最高的3个单词
top_three = word_counts.most_common(3)
print(top_three)
# Outputs [('eyes', 8), ('the', 5), ('look', 4)]

print( word_counts['not'])
print( word_counts['eyes'])

[('eyes', 8), ('the', 5), ('look', 4)]
1
8


In [19]:
morewords = ['why','are','you','not','looking','in','my','eyes']
for word in morewords:
    word_counts[word] += 1
print(word_counts['eyes'])
print(word_counts.update(morewords))

9
None


In [20]:
a = Counter(words)
b = Counter(morewords)
c = a + b
print(c)
d = a - b
print(d)

Counter({'eyes': 9, 'the': 5, 'look': 4, 'my': 4, 'into': 3, 'not': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1, 'why': 1, 'are': 1, 'you': 1, 'looking': 1, 'in': 1})
Counter({'eyes': 7, 'the': 5, 'look': 4, 'into': 3, 'my': 2, 'around': 2, "don't": 1, "you're": 1, 'under': 1})


# 毫无疑问， Counter 对象在几乎所有需要制表或者计数数据的场合是非常有用的工具。 在解决这类问题的时候你应该优先选择它